In [4]:
import numpy as np
import SimpleITK as sitk

from itkwidgets import view
from scipy.ndimage import zoom
from scipy.io import savemat, loadmat
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm as tqdm
sys.path.append('Packages')
from data import convert

%matplotlib widget

## Gaussian random field generation

In [5]:
def fftIndgen(n):
    a = list(range(0, int(n/2+1)))
    b = list(range(1, int(n/2)))
    b = b[::-1]
    b = [-i for i in b]
    return a + b

def gaussian_random_field(Pk = lambda k : k**-3.0, size = 100):
    def Pk2(kx, ky):
        if kx == 0 and ky == 0:
            return 0.0
        return np.sqrt(Pk(np.sqrt(kx**2 + ky**2)))
    noise = np.fft.fft2(np.random.normal(size = (size, size)))
    amplitude = np.zeros((size,size))
    for i, kx in enumerate(fftIndgen(size)):
        for j, ky in enumerate(fftIndgen(size)):            
            amplitude[i, j] = Pk2(kx, ky)
    return np.fft.ifft2(noise * amplitude)

In [8]:
size, sample_num = 128, 200
grf_batch = np.zeros((sample_num,size,size))
for i in tqdm(range(sample_num)):
    alpha = -2.3
    out = gaussian_random_field(Pk = lambda k: k**alpha, size=size)
    grf_batch[i] = out.real

savemat(f'grf/grf_{abs(alpha)}_size{size}_num{sample_num}.mat', {"grf": grf_batch})

  0%|          | 0/200 [00:00<?, ?it/s]

In [9]:
plt.figure()
plt.imshow(grf_batch[0], interpolation='none')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …